In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from bs4 import BeautifulSoup
from bs4.element import Tag
import codecs
import bs4
import spacy
import PyPDF2
from sklearn.metrics import make_scorer
import nltk
import requests
import scipy
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize, pos_tag

In [2]:
from sklearn.model_selection import train_test_split
import pycrfsuite
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import RandomizedSearchCV

In [3]:
fname="D:/!!~ Christopher ~ !!/Internship\Medical/Annotated1/Diagnosis.txt.xml"
with codecs.open(fname, "r", "utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")
text=soup.prettify().lower()

In [4]:
def filter(words):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(words) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    sentence=' '.join(filtered_sentence)
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(sentence)
    return new_words

In [5]:
docs = []
for elem in soup.find_all("document"):
    texts = []
    labels=''
    for j in elem.find_all("medicalrecord"):
        for child in j:
            if child.name == "symptom":
                labels = "symptom"
                w=child.text.replace('\t','')
                w=child.text.replace('\n',' ')
                w1=filter(w)
                for i in w1:
                    texts.append((i.strip(), labels))
            elif child.name=="schema":
                labels= "schema"
                w=child.text.replace('\t','')
                w=child.text.replace('\n',' ')
                w1=filter(w)
                for i in w1:
                    texts.append((i.strip(), labels))
            elif child.name=="pastmedicalhistory":
                labels = "pastmedicalhistory"
                w=child.text.replace('\t','')
                w=child.text.replace('\n',' ')
                w1=filter(w)
                for i in w1:
                    texts.append((i.strip(), labels))
            elif child.name=="exposuretomedication":
                labels = "exposuretomedication"
                w=child.text.replace('\t','')
                w=child.text.replace('\n',' ')
                w1=filter(w)
                for i in w1:
                    texts.append((i.strip(), labels))
            elif child.name=="diagnosis":
                labels = "diagnosis"
                w=child.text.replace('\t','')
                w=child.text.replace('\n',' ')
                w1=filter(w)
                for i in w1:
                    texts.append((i.strip(), labels))
            elif child.name=="socialhistory":
                labels = "socialhistory"
                w=child.text.replace('\t','')
                w=child.text.replace('\n',' ')
                w1=filter(w)
                for i in w1:
                    texts.append((i.strip(), labels))
        docs.append(texts)

In [6]:
data = []
for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = pos_tag(tokens)
    j=[(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)]
    data.append(j)

In [7]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'word.lower=' + word.lower(),
        'postag=' + postag
    ]
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not 
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [8]:
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

def get_token(doc):
    return [token for (token, postag, label) in doc]


X = [get_token(doc) for doc in data]
y = [get_labels(doc) for doc in data]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 211
Seconds required: 0.033

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 37893.748839
Feature norm: 1.000000
Error norm: 23317.475340
Active features: 211
Line search trials: 1
Line search step: 0.000023
Seconds required for this iteration: 0.032

***** Iteration #2 *****
Loss: 19842.764125
Feature norm: 2.367394
Error norm: 26084.453277
Active features: 205
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.020

***** Iteration #3 *****
Loss: 18276.659442
Feature norm: 2.761685
Error norm: 27593.838133
Active features: 200
Line search trials: 2
Line search step: 0.500000
Seconds required for this it

***** Iteration #48 *****
Loss: 305.581826
Feature norm: 43.732376
Error norm: 265.876864
Active features: 205
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.021

***** Iteration #49 *****
Loss: 298.928074
Feature norm: 43.798762
Error norm: 210.852871
Active features: 207
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.039

***** Iteration #50 *****
Loss: 293.692220
Feature norm: 43.914482
Error norm: 175.860046
Active features: 207
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.046

***** Iteration #51 *****
Loss: 289.506119
Feature norm: 44.063999
Error norm: 200.776519
Active features: 207
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.036

***** Iteration #52 *****
Loss: 284.921787
Feature norm: 44.226101
Error norm: 193.830998
Active features: 206
Line search trials: 2
Line search step: 0.500000
Seconds required for thi

***** Iteration #90 *****
Loss: 168.844749
Feature norm: 56.151832
Error norm: 168.022274
Active features: 206
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.034

***** Iteration #91 *****
Loss: 166.790919
Feature norm: 56.286754
Error norm: 105.574718
Active features: 206
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.029

***** Iteration #92 *****
Loss: 165.961673
Feature norm: 56.850268
Error norm: 169.914742
Active features: 207
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.026

***** Iteration #93 *****
Loss: 163.681167
Feature norm: 57.054820
Error norm: 84.649415
Active features: 207
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.021

***** Iteration #94 *****
Loss: 162.777928
Feature norm: 57.324444
Error norm: 128.408751
Active features: 207
Line search trials: 1
Line search step: 1.000000
Seconds required for this

***** Iteration #132 *****
Loss: 133.923241
Feature norm: 62.560435
Error norm: 40.415445
Active features: 199
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.018

***** Iteration #133 *****
Loss: 133.873086
Feature norm: 62.602829
Error norm: 74.828455
Active features: 200
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.017

***** Iteration #134 *****
Loss: 132.990842
Feature norm: 62.707550
Error norm: 36.858472
Active features: 200
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.019

***** Iteration #135 *****
Loss: 132.830202
Feature norm: 62.758812
Error norm: 72.160715
Active features: 200
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.024

***** Iteration #136 *****
Loss: 132.092296
Feature norm: 62.869682
Error norm: 41.527533
Active features: 200
Line search trials: 1
Line search step: 1.000000
Seconds required for thi

***** Iteration #176 *****
Loss: 121.571898
Feature norm: 64.473948
Error norm: 18.049553
Active features: 190
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.033

***** Iteration #177 *****
Loss: 121.418196
Feature norm: 64.470388
Error norm: 21.436349
Active features: 190
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.038

***** Iteration #178 *****
Loss: 121.297811
Feature norm: 64.462053
Error norm: 15.200558
Active features: 189
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.050

***** Iteration #179 *****
Loss: 121.175677
Feature norm: 64.438445
Error norm: 19.798856
Active features: 189
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.051

***** Iteration #180 *****
Loss: 121.051329
Feature norm: 64.405873
Error norm: 10.540500
Active features: 188
Line search trials: 2
Line search step: 0.500000
Seconds required for thi

In [9]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

C:\Users\Dell\Proj\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [10]:
labels = list(crf.classes_)
print(labels)

['schema', 'symptom', 'pastmedicalhistory', 'socialhistory', 'diagnosis', 'exposuretomedication']


In [11]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=2,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=20,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 20 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   32.5s finished


RandomizedSearchCV(cv=2,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=True,
                                 keep_tempfiles=None, max_iterations=100),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000270BB9195F8>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000270BB9198D0>},
                   scoring=make_scorer(flat_f1_score, average=weighted, labels=['schema', 'symptom', 'pastmedicalhistory', 'socialhistory', 'diagnosis', 'exposuretomedication']),
                   verbose=1)

In [12]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

In [13]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

C:\Users\Dell\Proj\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass labels=['pastmedicalhistory', 'schema', 'diagnosis', 'socialhistory', 'exposuretomedication', 'symptom'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


                      precision    recall  f1-score   support

  pastmedicalhistory      1.000     1.000     1.000        88
              schema      1.000     1.000     1.000       280
           diagnosis      1.000     1.000     1.000      5960
       socialhistory      1.000     1.000     1.000        32
exposuretomedication      1.000     1.000     1.000        48
             symptom      1.000     1.000     1.000      4768

            accuracy                          1.000     11176
           macro avg      1.000     1.000     1.000     11176
        weighted avg      1.000     1.000     1.000     11176



In [14]:
y_pred

[['schema',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'pastmedicalhistory',
  'pastmedicalhistory',
  'pastmedicalhistory',
  'pastmedicalhistory',
  'pastmedicalhistory',
  'socialhistory',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'schema',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
 

In [15]:
sent=word_tokenize("Kidney")
y_pred = crf.predict(sent)
y_pred

[['pastmedicalhistory',
  'pastmedicalhistory',
  'pastmedicalhistory',
  'pastmedicalhistory',
  'pastmedicalhistory',
  'pastmedicalhistory']]

In [40]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred=tagger.tag(sent_tokenize("Gastroesophageal reflux disease"))
y_pred

['diagnosis']

In [17]:
data

[[('trg_Abdominalpain', 'JJ', 'schema'),
  ('Abdominal', 'NNP', 'symptom'),
  ('pain', 'NN', 'symptom'),
  ('Stool', 'NNP', 'symptom'),
  ('black', 'JJ', 'symptom'),
  ('color', 'NN', 'symptom'),
  ('Abdominal', 'NNP', 'symptom'),
  ('pain', 'NN', 'symptom'),
  ('Cachexia', 'NNP', 'symptom'),
  ('weakness', 'NN', 'symptom'),
  ('wasting', 'VBG', 'symptom'),
  ('body', 'NN', 'symptom'),
  ('Finger', 'NNP', 'symptom'),
  ('nail', 'NN', 'symptom'),
  ('clubbing', 'VBG', 'symptom'),
  ('Unintentional', 'JJ', 'symptom'),
  ('weight', 'NN', 'symptom'),
  ('loss', 'NN', 'symptom'),
  ('Anorexia', 'NNP', 'symptom'),
  ('Diarrhea', 'NNP', 'symptom'),
  ('Fatigue', 'NNP', 'symptom'),
  ('Unhealthy', 'NNP', 'symptom'),
  ('pale', 'JJ', 'symptom'),
  ('appearance', 'NN', 'symptom'),
  ('conjuctiva', 'VBD', 'symptom'),
  ('Family', 'NNP', 'pastmedicalhistory'),
  ('history', 'NN', 'pastmedicalhistory'),
  ('Severe', 'NNP', 'pastmedicalhistory'),
  ('flare', 'NN', 'pastmedicalhistory'),
  ('colitis'

In [18]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred=tagger.tag('Kidney'.split(' '))
y_pred

['pastmedicalhistory']

In [19]:
pdfFileObj = open('C:/Users/Dell/data/Current Medical Diagnosis & Treatment 2019.pdf', 'rb' )
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)  
'''for i in range(61,63):    
    pageObj = pdfReader.getPage(i) '''
pageObj = pdfReader.getPage(372)
pages=pageObj.extractText()
words=pages

In [20]:
words=words.strip('\n')
words=words.replace('\n-\n','')
words=words.replace('\n',' ')
separate = words.split('-')
combined = ''.join(separate)
words=combined

In [21]:
fwords=filter(words)

In [22]:
URL='https://www.practicefusion.com/blog/25-most-common-diagnoses/'
response=requests.get(URL).text
soup=BeautifulSoup(response,'html')

In [23]:
l = soup.find_all('ol')
s1=[]
for i in range(len(l)):
    s1.append(str(l[i].text))

In [24]:
res = [i.replace('\n', ' ') for i in s1] 

In [25]:
res1=' '.join(res)

In [26]:
res1=filter(res1)

In [27]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
symptom -> symptom 5.558158
pastmedicalhistory -> pastmedicalhistory 4.915279
socialhistory -> socialhistory 4.065055
diagnosis -> diagnosis 3.051513
exposuretomedication -> exposuretomedication 1.824092
pastmedicalhistory -> socialhistory 1.381318
pastmedicalhistory -> exposuretomedication -0.014925
exposuretomedication -> socialhistory -0.035319
socialhistory -> diagnosis -0.058038
symptom -> exposuretomedication -0.215710
schema -> symptom -0.319973
schema -> schema  -0.364469
exposuretomedication -> diagnosis -0.553050
pastmedicalhistory -> diagnosis -1.168560
socialhistory -> schema  -1.467722
symptom -> diagnosis -1.650075
socialhistory -> pastmedicalhistory -2.101800
pastmedicalhistory -> schema  -2.110650
schema -> exposuretomedication -2.345773
schema -> diagnosis -2.857988

Top unlikely transitions:
socialhistory -> pastmedicalhistory -2.101800
pastmedicalhistory -> schema  -2.110650
schema -> exposuretomedication -2.345773
schema -> diagnosis -2.85798

In [36]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred=tagger.tag(word_tokenize(words))
c=1
for i in y_pred:
    print(str(c)+' '+i)
    c+=1

1 diagnosis
2 diagnosis
3 diagnosis
4 diagnosis
5 diagnosis
6 diagnosis
7 diagnosis
8 diagnosis
9 diagnosis
10 pastmedicalhistory
11 pastmedicalhistory
12 pastmedicalhistory
13 pastmedicalhistory
14 symptom
15 symptom
16 symptom
17 symptom
18 symptom
19 symptom
20 symptom
21 symptom
22 symptom
23 symptom
24 symptom
25 symptom
26 symptom
27 symptom
28 symptom
29 symptom
30 symptom
31 symptom
32 symptom
33 symptom
34 symptom
35 symptom
36 symptom
37 symptom
38 symptom
39 symptom
40 symptom
41 symptom
42 symptom
43 symptom
44 symptom
45 symptom
46 symptom
47 symptom
48 symptom
49 symptom
50 symptom
51 symptom
52 symptom
53 symptom
54 symptom
55 symptom
56 symptom
57 symptom
58 symptom
59 symptom
60 symptom
61 symptom
62 symptom
63 symptom
64 symptom
65 symptom
66 symptom
67 symptom
68 symptom
69 symptom
70 symptom
71 symptom
72 symptom
73 symptom
74 symptom
75 symptom
76 symptom
77 symptom
78 symptom
79 symptom
80 symptom
81 symptom
82 symptom
83 symptom
84 symptom
85 symptom
86 symptom
8

In [35]:
c=1
for i in word_tokenize(words):
    print(str(c)+' '+i)
    c+=1

1 C
2 HA
3 P
4 TER
5 1
6 0
7 352
8 ˚
9 Clinical
10 Findings
11 A
12 .
13 Symptoms
14 and
15 Signs
16 Mitral
17 valve
18 prolapse
19 without
20 significant
21 mitral
22 regurgitation
23 is
24 usually
25 asymptomatic
26 but
27 may
28 be
29 associated
30 with
31 a
32 syndrome
33 of
34 nonspecific
35 chest
36 pain
37 ,
38 dyspnea
39 ,
40 fatigue
41 ,
42 or
43 palpitations
44 .
45 Most
46 patients
47 are
48 young
49 ,
50 female
51 ,
52 thin
53 ,
54 and
55 some
56 have
57 skeletal
58 deformities
59 ,
60 such
61 as
62 pectus
63 excavatum
64 or
65 scoliosis
66 .
67 On
68 auscultation
69 ,
70 there
71 are
72 characteristic
73 mid
74 systolic
75 clicks
76 that
77 may
78 be
79 multiple
80 and
81 emanate
82 from
83 the
84 chordae
85 or
86 redundant
87 valve
88 tissue
89 .
90 If
91 leaflets
92 fail
93 to
94 come
95 together
96 properly
97 ,
98 the
99 clicks
100 will
101 be
102 followed
103 by
104 a
105 late
106 systolic
107 murmur
108 .
109 As
110 the
111 mitral
112 regurgitation
113 worsens
114 ,


In [30]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred=tagger.tag(("Radicular pain and lower extremity weakness.").split(' '))
y_pred

['symptom', 'symptom', 'symptom', 'symptom', 'symptom', 'symptom']

In [31]:
y_pred = crf.predict(sent_tokenize("GERD"))
y_pred

[['diagnosis', 'diagnosis', 'diagnosis', 'diagnosis']]

In [32]:
y_pred = crf.predict("alcohol")
for i in y_pred:
    print(i)

['symptom']
['diagnosis']
['pastmedicalhistory']
['symptom']
['socialhistory']
['symptom']
['diagnosis']


In [33]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred=tagger.tag(sent_tokenize("Radicular pain and lower extremity weakness."))
y_pred

['diagnosis']

In [34]:
y

[['schema',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'pastmedicalhistory',
  'pastmedicalhistory',
  'pastmedicalhistory',
  'pastmedicalhistory',
  'pastmedicalhistory',
  'socialhistory',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'diagnosis',
  'schema',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
  'symptom',
 